## Comparing Encoding Schemes and Frameworks in Neural Net Training

This project explores the impact of different label encoding schemes — sparse encoding and one-hot encoding — on the performance of neural network models. Utilizing the popular MNIST dataset, we investigate how each encoding scheme affects model accuracy in two leading deep learning frameworks: Keras (with TensorFlow backend) and PyTorch.

### Keras implementation

In [1]:
# Dataset Loading and Preprocessing
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy

# Load dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize images: divide each pixel value in the images by 255.0, scaling it from 0-255 to 0.0-1.0
# Standardizing the input data helps in stabilizing and speeding up the training
train_images = train_images / 255.0
test_images = test_images / 255.0

# One-hot encode labels for one model
train_labels_one_hot = to_categorical(train_labels)
test_labels_one_hot = to_categorical(test_labels)



11490434/11490434 [==============================] - 4s 0us/step


In [3]:
# Model Definition

def build_model():
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(128, activation='relu'), # Flatten 28x28 images to a 784 vector for each image
        Dense(10) # 10 output classes
    ])
    return model

In [4]:
# Training with Sparse Labels
model_sparse = build_model()
model_sparse.compile(optimizer='adam',
                     loss=SparseCategoricalCrossentropy(from_logits=True),
                     metrics=['accuracy'])

model_sparse.fit(train_images, train_labels, epochs=5)
model_sparse.evaluate(test_images, test_labels)



Epoch 1/5


1875/1875 [==============================] - 7s 3ms/step - loss: 0.2594 - accuracy: 0.9258
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1156 - accuracy: 0.9657
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0805 - accuracy: 0.9753
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0591 - accuracy: 0.9817
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0790 - accuracy: 0.9762


[0.079033263027668, 0.9761999845504761]

In [5]:
# Training with One-Hot Encoded Labels
model_one_hot = build_model()
model_one_hot.compile(optimizer='adam',
                      loss=CategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])

model_one_hot.fit(train_images, train_labels_one_hot, epochs=5)
model_one_hot.evaluate(test_images, test_labels_one_hot)

Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2612 - accuracy: 0.9255
Epoch 2/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.1152 - accuracy: 0.9665
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0799 - accuracy: 0.9758
Epoch 4/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.0609 - accuracy: 0.9814
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 0.0745 - accuracy: 0.9766


[0.07450196892023087, 0.9765999913215637]

### PyTorch implementation

In [6]:
# Dataset Loading and Preprocessing

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Transformations
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load dataset and prepare DataLoaders for batching the images
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False)


1.0%

100.0%


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



100.0%

Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw




9.9%

100.0%


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



100.0%

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [10]:
# Model Definition

# A basic feedforward neural network suitable for datasets like MNIST
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 128) # Flatten 28x28 images to a 784 vector for each image
        self.fc2 = nn.Linear(128, 10) # 10 output classes

    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
# Initialize the model
model = Net()


In [8]:
# Training and evaluation function for both Sparse and One-Hot encoded labels

def train_and_evaluate(model, trainloader, testloader, epochs=5, encoding='sparse'):
    if encoding == 'sparse':
        criterion = torch.nn.CrossEntropyLoss()
    elif encoding == 'one_hot':
        criterion = torch.nn.BCEWithLogitsLoss()  # Assuming binary classification for simplicity

    optimizer = torch.optim.Adam(model.parameters())

    for epoch in range(epochs):
        model.train()
        for inputs, labels in trainloader:
            optimizer.zero_grad()

            outputs = model(inputs)

            if encoding == 'one_hot':
                labels = torch.nn.functional.one_hot(labels, num_classes=10).float()  # Adjust num_classes as needed

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # Evaluation
        correct = 0
        total = 0
        model.eval()
        with torch.no_grad():
            for inputs, labels in testloader:
                outputs = model(inputs)
                if encoding == 'sparse':
                    _, predicted = torch.max(outputs.data, 1)
                elif encoding == 'one_hot':
                    predicted = outputs.argmax(dim=1, keepdim=True).flatten()

                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f'Epoch {epoch+1}, Encoding {encoding}: Accuracy {100 * correct / total:.2f}%')

    return model



In [11]:
# Training with Sparse Labels
train_and_evaluate(model, trainloader, testloader, epochs=5, encoding='sparse')

Epoch 1, Encoding sparse: Accuracy 93.13%
Epoch 2, Encoding sparse: Accuracy 94.90%
Epoch 3, Encoding sparse: Accuracy 96.20%
Epoch 4, Encoding sparse: Accuracy 96.87%
Epoch 5, Encoding sparse: Accuracy 96.97%


Net(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [12]:
# Training with One-Hot labels
train_and_evaluate(model, trainloader, testloader, epochs=5, encoding='one_hot')

Epoch 1, Encoding one_hot: Accuracy 97.29%
Epoch 2, Encoding one_hot: Accuracy 97.22%
Epoch 3, Encoding one_hot: Accuracy 97.69%
Epoch 4, Encoding one_hot: Accuracy 97.73%
Epoch 5, Encoding one_hot: Accuracy 97.66%


Net(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

### Keras vs. PyTorch Models Comparison
* Keras: Both encoding schemes resulted in high accuracy, with one-hot encoding showing a slight edge. Training and evaluation iplementation though is easier.
* PyTorch: Similarly, one-hot encoding in PyTorch marginally outperformed sparse encoding, but the difference is minimal.